In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import Counter
import numpy as np

In [ ]:
initial_alignment = AlignIO.read("tcbfprs1.aln", "clustal")
num_sequences = len(initial_alignment)
alignment_length = initial_alignment.get_alignment_length()

In [ ]:
alignment_length


190

In [ ]:
def get_cons_scores(initial_alignment, alignment_length, residue_number = 0):
  conservation_scores = []

  for i in range(alignment_length):
      column = [seq[i] for seq in initial_alignment]
      residue_counts = Counter(column)
      most_frequent_residue = residue_counts.most_common(5)[residue_number][0]
      identical_count = column.count(most_frequent_residue)
      conservation_score = identical_count / num_sequences * 100
      conservation_scores.append((most_frequent_residue, conservation_score))
  return conservation_scores

In [ ]:
residue_number = 0
conservation_scores = get_cons_scores(initial_alignment, alignment_length, residue_number)

In [ ]:
conservation_scores

[('M', 95.45454545454545),
 ('K', 91.41414141414141),
 ('E', 20.707070707070706),
 ('-', 70.2020202020202),
 ('Y', 91.41414141414141),
 ('E', 97.47474747474747),
 ('L', 92.42424242424242),
 ('P', 29.797979797979796),
 ('R', 57.57575757575758),
 ('R', 88.38383838383838),
 ('T', 30.808080808080806),
 ('P', 88.38383838383838),
 ('V', 79.7979797979798),
 ('I', 64.64646464646465),
 ('I', 66.16161616161617),
 ('R', 96.46464646464646),
 ('I', 54.04040404040404),
 ('D', 98.98989898989899),
 ('G', 95.45454545454545),
 ('K', 53.535353535353536),
 ('A', 56.060606060606055),
 ('F', 94.94949494949495),
 ('H', 91.91919191919192),
 ('T', 75.75757575757575),
 ('F', 69.1919191919192),
 ('T', 94.44444444444444),
 ('R', 60.1010101010101),
 ('G', 78.28282828282829),
 ('K', 74.74747474747475),
 ('P', 98.48484848484848),
 ('F', 93.93939393939394),
 ('D', 95.95959595959596),
 ('E', 40.909090909090914),
 ('V', 27.27272727272727),
 ('L', 73.23232323232324),
 ('I', 27.77777777777778),
 ('K', 20.707070707070706)

In [ ]:
len(conservation_scores)

190

In [ ]:
cdd_433942 = "IVVRIDGRDFHRFSKVHEFEKPNDETALNLMNSCASAVLEEFPDIVFAYGYSDEYSFVFKKSTELFKRRASKILSLVASFFASVYVTKWKKFFPHLKLLYAPSFDGRVVSYPSVQVLKDYLAWRQVDCHINNLYNTCFWMLVKKSGKTPSQAQEILKGTFSAEKNEILFSEFGINYNNEPEMFRKGSILIRK"
cdd_427953 = "YEYVKSFEQDDKLLPNTWIVVRIDGRGFHKFSKKHGFEKPNDERALDLMNEAAKDVMEEFPDIVLAYGQSDEYSFVFRKSTTLFNRRVSKLVSLVASLFSSSYVFLWSKFFPDKPLKYPPSFDGRAVLYP"
cdd_226508 = "DRILPQTYIVLRIDGRGFHKFTKFLDFEKPYDERFLKLMNATAKNLVLKYGLDIILAYTFSDEISFLLKSSTVPFNGRVEKLDSVFASFFTSAFTRLWAKFFPEKHLPSFDSRCVAYPLDTIPDYFHWRQVEAWRNNLYSTTFWQLIIRGLTPQEAEERLRGTKSNEKHEILFSEFGINYNREPEWQKKG"

In [ ]:
alignment_consensus = ""
for residue_pair in conservation_scores:
  alignment_consensus += residue_pair[0]
alignment_consensus


alignment_consensus

'MKE-YELPRRTPVIIRIDGKAFHTFTRGKPFDEVLIKAMQETMKYLCENIQGCVLGYTQSDEISLLLTDTQAWFDYNVQKMVSVSASMATAAFNAMFDARAFNIPK-EEVTNYFLWRQQDATRNSIQMVGQANFSHKELQGKS-QDMLMTEKGINWNDLPTWQKRGSCCIKWIIDKEIPIFKREYIEQLV'

In [ ]:
def get_cons_aln(initial_alignment, alignment_length, conservation_scores, threshold = 80):
  threshold = threshold
  conserved_coordinates = []
  conserved_regions = []

  for i in range(alignment_length):
      if conservation_scores[i][1] >= threshold:
          conserved_coordinates.append(i)
          conserved_regions.append(alignment_consensus[i])
  conserved_alignment = []

  for record in initial_alignment:
      conserved_sequence = "".join([record.seq[i] for i in conserved_coordinates])
      conserved_record = SeqRecord(Seq(conserved_sequence), id=record.id)
      conserved_alignment.append(conserved_record)

  conserved_alignment = MultipleSeqAlignment(conserved_alignment)
  return (conserved_alignment, conserved_coordinates)

In [ ]:
conservative_statistics = {}
for threshold in np.array((80, 90, 95, 97, 99, 100)):
    conservative_statistics[f"Threshold{threshold}"] = (get_cons_aln(initial_alignment, alignment_length, conservation_scores, threshold)[0].get_alignment_length(),
                                                        np.array(get_cons_aln(initial_alignment, alignment_length, conservation_scores, threshold)[1]),
                                                        get_cons_aln(initial_alignment, alignment_length, conservation_scores, threshold)[0])

conservative_statistics


{'Threshold80': (73,
  array([  0,   1,   4,   5,   6,   9,  11,  15,  17,  18,  21,  22,  25,
          29,  30,  31,  41,  45,  51,  56,  58,  59,  60,  61,  62,  66,
          72,  73,  78,  79,  82,  85,  86,  88,  89,  92,  93,  94,  96,
          97,  99, 104, 106, 108, 109, 111, 115, 116, 117, 119, 120, 123,
         124, 125, 130, 133, 138, 143, 147, 154, 155, 156, 159, 163, 164,
         165, 167, 171, 174, 178, 180, 182, 184]),
  <<class 'Bio.Align.MultipleSeqAlignment'> instance (198 records of length 73) at 7ed8572190c0>),
 'Threshold90': (55,
  array([  0,   1,   4,   5,   6,  15,  17,  18,  21,  22,  25,  29,  30,
          31,  41,  45,  51,  56,  58,  59,  60,  61,  62,  66,  73,  79,
          82,  85,  86,  88,  89,  92,  94,  96,  97,  99, 104, 106, 108,
         109, 111, 115, 116, 117, 119, 123, 124, 143, 147, 155, 164, 165,
         171, 174, 178]),
  <<class 'Bio.Align.MultipleSeqAlignment'> instance (198 records of length 55) at 7ed83829a590>),
 'Threshold95': (

In [ ]:
conservative_statistics["Threshold97"]

(20,
 array([  5,  17,  29,  58,  59,  60,  61,  73,  79,  82,  92,  96,  97,
         99, 115, 116, 117, 119, 165, 178]),
 <<class 'Bio.Align.MultipleSeqAlignment'> instance (198 records of length 20) at 78624bd363b0>)

In [ ]:
string = "MKEYELPRRTPVIIRIDGKAFHTFTRGKPFDEVLIKAMQETMKYLCENIQGCVLGYTQSDEISLLLTDTQAWFDYNVQKMVSVSASMATAAFNAMFDARAFNIPKEEVTNYFLWRQQDATRNSIQMVGQANFSHKELQGKSQDMLMTEKGINWNDLPTWQKRGSCCIKWIIDKEIPIFKREYIEQLV"

**Соответствие консервативного анализа**:

**100%**: 59 (S59), 61 (E61), 79 (K79), 99 (R99)

**99%**: 60 (D60), 96 (F), 97 (D), 119 (D)

**97%**: 5 (E6), 17 (D17), 29, 58, 73, 82, 92, 115, 116, 117, 165, 178

---



In [ ]:
counter = 0
for i in range(len(conservation_scores)):
  if 100 > conservation_scores[i][1] >= 99:
    print(conservation_scores[i], counter)
  counter += 1

('D', 99.4949494949495) 60
('F', 99.4949494949495) 96
('D', 99.4949494949495) 97
('D', 99.4949494949495) 119


In [ ]:
def get_cons_scores_2(initial_alignment, alignment_length, residue_number = 0):
  conservation_scores_2 = []

  for i in range(alignment_length):
      column = [seq[i] for seq in initial_alignment]
      residue_counts = Counter(column)
      if len(residue_counts.most_common(5)) > 1:
        most_frequent_residue = residue_counts.most_common(5)[residue_number][0]
      else:
        most_frequent_residue = "None"
      identical_count = column.count(most_frequent_residue)
      conservation_score_2 = identical_count / num_sequences * 100
      conservation_scores_2.append((most_frequent_residue, conservation_score_2))
  return conservation_scores_2

In [ ]:
residue_number = 1
conservation_scores_2 = get_cons_scores_2(initial_alignment, alignment_length, residue_number)

In [ ]:
conservation_scores_2

[('-', 2.0202020202020203),
 ('R', 6.0606060606060606),
 ('G', 18.181818181818183),
 ('F', 13.636363636363635),
 ('G', 5.555555555555555),
 ('-', 2.0202020202020203),
 ('-', 2.0202020202020203),
 ('T', 25.757575757575758),
 ('K', 14.14141414141414),
 ('G', 6.0606060606060606),
 ('M', 25.252525252525253),
 ('W', 4.040404040404041),
 ('I', 6.0606060606060606),
 ('V', 12.121212121212121),
 ('L', 20.707070707070706),
 ('-', 1.5151515151515151),
 ('L', 27.77777777777778),
 ('-', 1.0101010101010102),
 ('M', 3.0303030303030303),
 ('R', 21.21212121212121),
 ('H', 23.232323232323232),
 ('G', 3.535353535353535),
 ('S', 6.0606060606060606),
 ('S', 12.626262626262626),
 ('Y', 22.22222222222222),
 ('C', 2.0202020202020203),
 ('K', 31.313131313131315),
 ('N', 4.040404040404041),
 ('R', 21.71717171717172),
 ('-', 1.0101010101010102),
 ('Y', 5.05050505050505),
 ('S', 2.0202020202020203),
 ('D', 15.151515151515152),
 ('I', 23.737373737373737),
 ('F', 18.68686868686869),
 ('M', 16.666666666666664),
 ('N

In [ ]:
conservation_scores[28]

('K', 74.74747474747475)

In [ ]:
conservation_scores_2[28]

('R', 21.71717171717172)

In [ ]:
conservation_scores[59]

('S', 100.0)

In [ ]:
conservation_scores[126]

('Q', 46.96969696969697)

In [ ]:
conservation_scores_2[126]

('S', 30.808080808080806)

In [ ]:
counter = 0
for i in range(len(conservation_scores_2)):
  if 1 > conservation_scores_2[i][1] > 0:
    print(conservation_scores_2[i], counter)
  counter += 1

('N', 0.5050505050505051) 60
('L', 0.5050505050505051) 96
('N', 0.5050505050505051) 97
('A', 0.5050505050505051) 115
('K', 0.5050505050505051) 116
('G', 0.5050505050505051) 119
('E', 0.5050505050505051) 165


('D', 99.4949494949495) 60

('F', 99.4949494949495) 96

('D', 99.4949494949495) 97

('D', 99.4949494949495) 119

Последовательность, в которой замена D -> N в активном центре

In [ ]:
from Bio import AlignIO

# Load the alignment
initial_alignment = AlignIO.read("tcbfprs1.aln", "clustal")

# Define the position and residue you're looking for
position = 60  # 0-based index
residue = 'N'  # e.g., Asn

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: MKTFYELMRRCPVAIRIDGKAFHTFTRGKPFDEVLIKSMQQTMKYLCENIQGCVLGYTQSNEITLILVDSSAWFDYEVQKMCSIAASMTTMAFNAMFDARCFNIPK-EEVTNLVYWRQLDASRNSIQMLGQANFSHSELQNKS-QDMLMMQKGINWNDLPVYQKRGSCCVRWIVDKNIPIFRRNYIDKLI
Name: WP_118197436.1
Description: WP_118197436.1



In [ ]:
# Define the position and residue you're looking for
position = 17  # 0-based index
residue = '-'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: --------------------------------------------------MGCQLIYHQSDEISLLLTNSESWFKNNLQKIASVSASMATAKFNAIFDSRAWVLPP-DEVMNYFIWRQKDATKNSISMLAQNHFSQGDLTGLD-QDKLMKEKGVNWNDLPIWKKRGVCIRKWEVDHDTPIFTRTYINQFI
Name: WP_283770932.1
Description: WP_283770932.1

Sequence: -------------------------------------------------MMGCKLIYHQSDEISILLTNTQSWFENNLQKMVSVSASMATAKFNATFDSRAWVLPH-DEVTNYFLWRQQDATKNSISMVAQANFPHEKLQGLD-QDKLFLEKGINWNDLPVWQKRGACITKWDVDLNTPIFSREYINQYV
Name: GJM71831.1
Description: GJM71831.1



In [ ]:
len('MRALEFMPGVFAVARVDGRSFTRLTKEAPFDLRFSDYMAATALHLCNCGFQAIYAYTQSDEISLLFRAEDETFQRKLRKWNSILASEAGAVFSAAFDCRIIPLPNKKSVHSRPTAKQKGDWRPTPHRVGLR')

131

In [ ]:
# Define the position and residue you're looking for
position = 28  # 0-based index
residue = 'E'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: MKG-YELTRRTPVILRVDGRAFHTFTRGEPFDAILHNAMVATANALVKEIQGAKVAYGQSDEISVLITDTDAWFGYGIQKMVSIAASVATIVFNAQFDARVFNLPK-EDVVNYFVWRQQDATRNSIQMVARCYFSHGQCHKKN-QEMLMSECRVNWNDTPTHFKRGFCVVG-TPDREIPIFTREYLERFV
Name: WP_297653719.1
Description: WP_297653719.1



In [ ]:
seq = 'MKEFYELTRRTPVILRLDGKSFHTYCKSDRFNKGFMRCMDLTAIRLMGEIQGAQLAFVQSDEISILLHDSEAWMKNNIQKMVSISAAIASVEFTAYFDSRVFTIPE-AEVCNYFIWRQNDATRNSILSLGQSLFSHKQLQGLN-QDICH-DKGHNWNDLPTSHKRGRCIVKWVVDEEIPIFTRNYINKYL'
seq[126]


'L'

In [ ]:
# Define the position and residue you're looking for
position = 115  # 0-based index
residue = 'A'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: MRA-LEFMPGVFAVARVDGRSFTRLTKEAPFDLRFSDYMAATALHLCNCGFQAIYAYTQSDEISLLFRAEDETFQRKLRKWNSILASEAGAVFSAAFDCRIIPLPNKKSVHSRPTAKQKGDWRPTPH--RVGLR--------------------------------------------------------
Name: WP_198141211.1
Description: WP_198141211.1



In [ ]:
# Define the position and residue you're looking for
position_1 = 28  # 0-based index
residue_1 = 'R'  # e.g., Alanine
position_2 = 127  # 0-based index
residue_2 = 'N'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if str(seq[position_1]) == residue_1 and seq[position_2] == residue_2:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print(f'Length: {len(seq.replace("-", ""))}')
        print()  # empty line for readability

Sequence: MKEFYELPRRTFTIIRIDGKAFHTYTRGRPFDEPFMRDMDETAKFLCANIQGARFAFVQSDEISVLLTDTAAWFDGNIQKMVSISASLATARFNAFFDSRVFSIPSAVEVENYFIWRQQDATRNSISNVAQSLYSHKELNGKT-QEMIF-QKGINWNDFPPRCKRGRLIVKWTV-ADVPIFTREFLQTLM
Name: MCU0355397.1
Description: MCU0355397.1
Length: 187

Sequence: MKMYYELPRRTYTIIRIDGKAFHTYTRGRPFDDQFMQDMDATASFLCANIQGARFGYVQSDEISIALTDTAAWFDGNIQKMVSISASLATAKFNAYFDSRVFTIPFQIEVENYFIWRQQDATRNSISNVAQSLYTHKELTNKS-QELIF-QKGINWNDYAPRYKRGRIITKWTV-SDVPIFTRGFLQKLL
Name: WP_313998006.1
Description: WP_313998006.1
Length: 187



In [ ]:
# Define the position and residue you're looking for
position = 119  # 0-based index
residue = 'G'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: MRA-LEFMPGVFAVARVDGRSFTRLTKEAPFDLRFSDYMAATALHLCNCGFQAIYAYTQSDEISLLFRAEDETFQRKLRKWNSILASEAGAVFSAAFDCRIIPLPNKKSVHSRPTAKQKGDWRPTPH--RVGLR--------------------------------------------------------
Name: WP_198141211.1
Description: WP_198141211.1



In [ ]:
# Define the position and residue you're looking for
position = 165  # 0-based index
residue = 'E'  # e.g., Alanine

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    if seq[position] == residue:
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability

Sequence: MKG-YELPRRMPVIIRIDGKAFHTYTRGKPFDEYLSDAMWGTCVYLAQNIMGCKLAYTQSDEISLLLTNTEAWFDNNLQKLVSVSTSLATAKFNALFDARAWVLPK-DEVCNYFLWRQQDATKNSISMVAQANFPHKQLQGLN-QDKLYLEKEINWNDLPTWQKREACIKKWDVDFETPIFSRDYVEQYV
Name: WP_196616409.1
Description: WP_196616409.1



Description: WP_118197436.1

Description: RKI83929.1

Description: MEE3499559.1

Description: WP_198141211.1

Description: WP_196616409.1

In [ ]:
# Define the position and residue you're looking for
position_1 = 17  # 0-based index
residue_1 = 'D'  # e.g., Alanine
position_2 = 60  # 0-based index
residue_2 = 'D'  # e.g., Alanine
position_3 = 61  # 0-based index
residue_3 = 'E'  # e.g., Alanine

position_4 = 28  # 0-based index
residue_4 = 'K'  # e.g., Alanine
position_5 = 59  # 0-based index
residue_5 = 'S'  # e.g., Alanine
position_6 = 127  # 0-based index
residue_6 = 'M'  # e.g., Alanine

position_7 = 15  # 0-based index
residue_7 = 'R'  # e.g., Alanine
position_8 = 79  # 0-based index
residue_8 = 'K'  # e.g., Alanine
position_9 = 99  # 0-based index
residue_9 = 'R'  # e.g., Alanine

position_10 = 5  # 0-based index
residue_10 = 'E'  # e.g., Alanine
position_11 = 116  # 0-based index
residue_11 = 'R'  # e.g., Alanine


# Iterate over the sequences in the alignment
counter = 0
for record in initial_alignment:
    seq = str(record.seq)
    if (
        seq[position_1] == residue_1 and seq[position_2] == residue_2
        and seq[position_3] == residue_3 and seq[position_4] == residue_4
        and seq[position_5] == residue_5 and seq[position_6] == residue_6
        and seq[position_7] == residue_7 and seq[position_8] == residue_8
        and seq[position_9] == residue_9 and seq[position_10] == residue_10
        and seq[position_11] == residue_11):
        counter += 1
        print(f"Sequence: {seq}")
        print(f"Name: {record.id}")
        print(f"Description: {record.description}")
        print()  # empty line for readability
print(counter)

Sequence: MKG-YELPKRMPVIIRIDGKAFHTYTKEKPFDKDLTNAMWETCIYLAKNIMGCKLVYTQSDEISLLLTNTEAWFDNNLQKIVSVSASLATAKFNALFDARAWVLPK-DEVCNYFWWRQQDATKNSISMVAQANFSQKQLQGLN-QDKLFLEKGINWNDIPTWQKRGACIRKWDVDFETPIFSREYVEQYV
Name: WP_175076441.1
Description: WP_175076441.1

Sequence: MKA-YELPRRLPVLIRLDGCHFHTYTKGKPYDEGLIKAFWQTCTYLGQKIMGCQLIYHQSDEISLLLTNSESWFKNNLQKIASVSASMATAKFNAIFDSRAWVLPP-DEVMNYFIWRQKDATKNSISMLAQNHFSQGDLTGLD-QDKLMKEKGVNWNDLPIWKKRGVCIRKWEVDHDTPIFTRTYINQFI
Name: WP_191141365.1
Description: WP_191141365.1

Sequence: MKG-YELPQRMPVILRIDGCHFHTFTRGKPFDEQLTGALWETCKYLASNIMGCKLVYHQSDEISILLTNTQSWFENNIQKMVSVSASLATAKFNATFDSRAWVLPH-DEVTNYFLWRQQDATKNSISMVAQAHFAHEELQGLD-QDKLFLEKGINWNDLPVWQKRGVCITKWDVDHDTPVFSRDYINQYV
Name: WP_192701293.1
Description: WP_192701293.1

Sequence: MKA-YELTRRVPVAIRVDGKAFHTFTRGKPFDEVLSNTMQATMMKMCRQIQGCVFAYTQSDEITFILIDSDGWFNYRTDKMCSIAASMATMEFNAMFDARAFSIPK-EEVTNLIYWRQQDAMRNAVQMVGQAFYSHKELQGVN-KEMLLADKGIEWDKIPVKYQRGSCCVKWTIDNNISIFRREQIDKLV
Name: WP_195465562.1
Description: WP_195465562